In [19]:
from authenticate import authenticate_google_calendar 

In [20]:
import datetime as dt

In [21]:

def fetch_study_events(service, calendar_id="primary"):
    now = dt.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    # Calculate the start of the month
    start_of_month = (
        dt.datetime.utcnow().replace(day=1) - datetime.timedelta(days=1)
    ).replace(day=1).isoformat() + "Z"
    print("Fetching past month events")
    events_result = (
        service.events()
        .list(
            calendarId=calendar_id,
            timeMin=start_of_month,
            timeMax=now,
            maxResults=100,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
        print("No past month events found.")
        return []

    for event in events:
        start = event["start"].get("dateTime", event["start"].get("date"))
        print(start, event["summary"])

    return events



In [22]:
import datetime as dt
import pytz

def create_study_plan(service, subjects, num_days, calendar_id="primary"):
    
    START_TIME = dt.time(9, 0)
    END_TIME = dt.time(18, 30)
    
    def delete_overlapping_events(start_time, end_time):
        """Deletes events in the study plan that overlap with the given time range."""
        events_result = service.events().list(
            calendarId=calendar_id, 
            timeMin=start_time.isoformat(),
            timeMax=end_time.isoformat(), 
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])

        for event in events:
            if 'summary' in event and event['summary'] in subjects:
                service.events().delete(calendarId=calendar_id, eventId=event['id']).execute()
                print(f"Deleted overlapping event: {event['summary']} from {event['start']['dateTime']} to {event['end']['dateTime']}.")

    now = dt.datetime.now(pytz.timezone('Europe/London'))
    current_date = now.date()
    study_dates = [current_date + dt.timedelta(days=i) for i in range(num_days)]
    
    total_weight = sum(subjects.values())

    for study_date in study_dates:
        if study_date == current_date and now.time() > START_TIME:
            start_time = max(now, pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, START_TIME)))
        else:
            start_time = pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, START_TIME))

        end_time = pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, END_TIME))
        lunch_start = pytz.timezone('Europe/London').localize(dt.datetime.combine(study_date, dt.time(12, 00)))
        lunch_end = lunch_start + dt.timedelta(minutes=45)

        # Delete overlapping events in the study plan
        delete_overlapping_events(start_time, end_time)

        time_available = end_time - max(start_time, now)  # Calculate available study time

        num_blocks_available = int((time_available.total_seconds() / 60) / 45)  # Number of 40-minute blocks available + 5-minute break

        print(f"Total available blocks: {num_blocks_available}")

        # Check if lunch is still left in the day
        if lunch_start > now:
            num_blocks_available -= 1

        subject_blocks = {subject: int(weight / total_weight * num_blocks_available) for subject, weight in subjects.items()}
        
        remaining_blocks = num_blocks_available - sum(subject_blocks.values())
        
        if remaining_blocks > 0:
            # Add remaining blocks to the subject with the highest weight
            max_weight_subject = max(subjects, key=subjects.get)
            subject_blocks[max_weight_subject] += remaining_blocks

        print("Number of blocks per subject:")
        for subject, blocks in subject_blocks.items():
            print(f"{subject}: {blocks} blocks")
        
        for subject, blocks in subject_blocks.items():
            for _ in range(blocks):
                if start_time >= end_time:
                    break
                
                # Skip inserting block during lunch
                if lunch_start <= start_time < lunch_end:
                    start_time = lunch_end
                    continue
                
                event_end = start_time + dt.timedelta(minutes=40)
                if event_end > end_time:
                    break

                event = {
                    "summary": f"{subject}",
                    "description": "Study block generated by study planner - Bill Rimell.",
                    "start": {
                        "dateTime": start_time.isoformat(),
                        "timeZone": "Europe/London",
                    },
                    "end": {
                        "dateTime": event_end.isoformat(),
                        "timeZone": "Europe/London",
                    },
                }
                service.events().insert(calendarId=calendar_id, body=event).execute()
                print(f"Added {subject} study block from {start_time} to {event_end}.")

                start_time = event_end + dt.timedelta(minutes=5)  # Next event starts after a gap
        
        # Add lunch block if it's not already added and lunch time is within the available study time
        if lunch_start < end_time and lunch_end > start_time:
            lunch_end = min(lunch_end, end_time)
            lunch_event = {
                "summary": "Lunch",
                "description": "Lunch break generated by study planner - Bill Rimell.",
                "start": {
                    "dateTime": lunch_start.isoformat(),
                    "timeZone": "Europe/London",
                },
                "end": {
                    "dateTime": lunch_end.isoformat(),
                    "timeZone": "Europe/London",
                },
            }
            service.events().insert(calendarId=calendar_id, body=lunch_event).execute()
            print(f"Added lunch break from {lunch_start} to {lunch_end}.")


In [23]:
calendar_id = "0ca09266015f691eebe0d00c6f3ed7a784713e0160a694b8f7929add00cb1aa1@group.calendar.google.com"

In [24]:
service = authenticate_google_calendar()

subjects = {'Maths': 150, 'Physics': 100, 'Computer Science': 70}  # Example subjects with weights
create_study_plan(service=service, subjects=subjects, num_days=1, calendar_id=calendar_id)  # Specify your calendar ID


Total available blocks: 1
Number of blocks per subject:
Maths: 1 blocks
Physics: 0 blocks
Computer Science: 0 blocks
Added Maths study block from 2024-04-11 17:36:13.085712+01:00 to 2024-04-11 18:16:13.085712+01:00.
